In [1]:
import os
import numpy as np
from tqdm import tqdm

In [2]:
positive_samples = np.load ( "../datasets/train/out_pos/data.mfcc.npy" )
negative_samples = np.load ( "../datasets/train/out_neg/data.mfcc.npy" )

In [3]:
#check balanceness of the data.
print(len(positive_samples))
print(len(negative_samples))

34524
34524


In [4]:
positive_samples.shape

(34524, 13, 87)

In [5]:
N_FEATURES = np.concatenate(positive_samples[2]).shape[0]

In [6]:
training_data = []
for i in range(positive_samples.shape[0]):
    training_data.append([np.concatenate(positive_samples[i]),np.eye(2)[0]]) 

for i in range(negative_samples.shape[0]):
    training_data.append([np.concatenate(negative_samples[i]),np.eye(2)[1]])

print(len(training_data))

69048


In [7]:
import random
random.shuffle(training_data) # shuffle the data!

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else  :
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = 16
        self.c2 = 32
        self.c3 = 64
        
        self.conv1 = nn.Conv1d(1,self.c1,5)
        self.conv2 = nn.Conv1d(self.c1,self.c2,5)
        self.conv3 = nn.Conv1d(self.c2,self.c3,5)
        
        x = torch.randn(N_FEATURES).view(-1,1,N_FEATURES)
        self._to_linear = None
        
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,2)
        
        
    def convs ( self, x ):
        
        x = F.max_pool1d(F.relu(self.conv1(x)),2)
        x = F.max_pool1d(F.relu(self.conv2(x)),2)
        x = F.max_pool1d(F.relu(self.conv3(x)),2)
        
        if self._to_linear is None:
            #print(x[0].shape)
            self._to_linear = x[0].shape[0]*x[0].shape[1]
        
        return x
    def forward(self,x):
        x = self.convs(x)
        
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x,dim=1)

In [15]:
net = ConvNet().to(device)

In [13]:
import torch.optim as optim
from sklearn.utils import shuffle
import random

X = torch.Tensor([i[0] for i in training_data]).view(-1,N_FEATURES)
max_training = torch.max(X)
min_training = torch.min(X)
X = (X-min_training)/ (max_training - min_training) * 2 -1 # scale between -1 and 1.

y = torch.Tensor( [i[1] for i in training_data])

VAL_PCT = 0.1

val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]
print(len(train_X))
print(len(test_X))

6904
62144
6904


In [16]:
train_X[0]

tensor([-0.2323, -0.2519, -0.2969,  ...,  0.4599,  0.4583,  0.4604])

In [17]:
train_X[0]

tensor([-0.2323, -0.2519, -0.2969,  ...,  0.4599,  0.4583,  0.4604])

In [18]:
BATCH_SIZE = 100

EPOCHS = 10
optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()
def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,N_FEATURES).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)

            net.zero_grad() # check why 
            outputs = net(batch_X)
            #print(batch_y.shape)
            loss  = loss_function(outputs,batch_y)
            loss.backward()
            optimizer.step()# check why 
        print(f"Epoch: {epoch}. Loss: {loss}")

train(net)

  1%|▏         | 8/622 [00:00<00:08, 75.44it/s]

Epoch: 0. Loss: 0.24956390261650085


  1%|▏         | 8/622 [00:00<00:07, 78.91it/s]

Epoch: 1. Loss: 0.24959252774715424


  1%|▏         | 8/622 [00:00<00:07, 79.90it/s]

...

Epoch: 7. Loss: 0.24961502850055695


  1%|▏         | 8/622 [00:00<00:08, 75.01it/s]

Epoch: 8. Loss: 0.2496185302734375


100%|██████████| 622/622 [00:08<00:00, 75.25it/s]

Epoch: 9. Loss: 0.2496190071105957


In [19]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i])
            net_out = net(test_X[i].view(-1,1,N_FEATURES).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print(f"Accuracy: {round(correct/total,3)}")
    
test(net)

100%|██████████| 6904/6904 [00:08<00:00, 775.05it/s]

Accuracy: 0.495


In [33]:
def fwd_pass(X,y,train=False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches = [ torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs,y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs,y)
    
    if train:
        loss.backward()
        optimizer.step()
    
    return acc, loss

In [34]:
def test (size=32):
    random_start = np.random.randint(len(test_X)-size)
    
    X,y = test_X[random_start:random_start+size], test_y [random_start:random_start+size]
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.view(-1,1,N_FEATURES).to(device),y.to(device))
    
    return val_acc,val_loss
    
val_acc, val_loss = test(size=100)
print(val_acc,val_loss)

0.55 tensor(0.2474, device='cuda:0')


In [36]:
import time

MODEL_NAME = f"model-{int(time.time())}.log"

net = ConvNet().to(device)

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()


print(MODEL_NAME)

def train():
    BATCH_SIZE = 100
    EPOCHS = 30
    with open(MODEL_NAME,"a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,N_FEATURES).to(device)
                batch_y = train_y[i:i+BATCH_SIZE].to(device)
                
                
                acc, loss = fwd_pass (batch_X,batch_y,train=True)
                if i % 50 == 0:
                    val_acc, val_loss = test(size=64)
                    f.write(f"{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)},{round(float(loss),4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n")
                

model-1592385501.log


In [37]:
train()

100%|██████████| 72/72 [00:01<00:00, 47.95it/s]


In [38]:
%matplotlib qt
import matplotlib.pyplot as plt
from matplotlib import style

style.use("ggplot")

def create_acc_loss_graph():
    contents = open(MODEL_NAME,"r").read().split('\n')
    
    times = []
    accuracies = []
    losses = []
    
    val_accs = []
    val_losses = []
    
    for c in contents:
        if MODEL_NAME in c:
            name,timestamp,acc,loss,val_acc,val_loss = c.split(",")
            
            times.append(float(timestamp))
            accuracies.append(float(acc))
            losses.append(float(loss))
            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))
            
    fig = plt.figure()
    ax1 = plt.subplot2grid((2,1),(0,0))
    ax2 = plt.subplot2grid((2,1),(1,0),sharex=ax1)
    
    ax1.plot(times,accuracies,label="acc")
    ax1.plot(times,val_accs,label="val_acc")
    ax1.legend(loc=2)
    
    ax2.plot(times,losses,label="loss")
    ax2.plot(times,val_losses,label="val_loss")
    ax2.legend(loc=2)
    
    plt.show()
create_acc_loss_graph()